# Transformers from scratch in pytorch
> Notes from https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html

- layout: post
- toc: false
- comments: false
- hide: false
- search_exclude: true
- categories: [fastpages, jupyter]


In [7]:
## Standard Libraries
import os
from tracemalloc import Snapshot
import numpy as numpy
import random
import math
import json
from functools import partial

# imports for plotting
import matplotlib.pyplot as plt
import matplotlib_inline
plt.set_cmap('cividis')
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats()
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

from tqdm.notebook import tqdm

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# path to folder where datasets should be downloaded
DATASET_PATH = "../data"
CHECKPOINT_PATH = "../saved_models"

# setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministirc on GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device: ", device)


Global seed set to 42


Device:  cpu


### What is Attention?
*The attention mechanism describes a weighted average of sequence of elements with the weights dynamically computed based on the input query and the element's keys.*

![image](http://jalammar.github.io/images/t/self-attention_softmax.png)


(image from http://jalammar.github.io/illustrated-transformer/)
#### Self-attention
We use self-attention to modify each embedding of the input word as a combination of modified `values` from the word embedding and the weights computed. Some intuitions
 - `Values` are nothing but modified word embeddings
 - `Query` is the modified word embedding which will search for relevant word embeddings in the sequence
 - `Keys` are modified word embeddings which will be searched by the query. 

Lets look at the statement `The animal didn't cross the street because it was too tired`. 

